In [ ]:
!pip install wikipedia

In [1]:
import requests
import wikipedia
import pandas as pd
import numpy as np

import re

In [2]:
def category_format( category):
    category_query = re.sub( '\s', '+', category)
    return category_query
    

In [3]:
def format_query(category, ptype ):
    '''Category should be provided as a string,  ptype may be page, subcat, or file'''
    categoryF = category_format(category)
    
    ptype_dict = {'page':'0', 'subcat':'14','file':'6'}
    nstype = ptype_dict[ ptype]
    #if ptype == 'page':
    #    nstype = '0'
    #if ptype == 'subcat':
    #    nstype = '14'
    #if ptype == 'file':
    #    nstype = '6'
    base_url = 'https://en.wikipedia.org/w/api.php'
    action_tag = "?action=query&list=categorymembers&cmlimit=max" ## fetch all category members (pages, subcategories)
    category_tag = '&cmtitle=Category:{}&cmtype={}&cmnamespace={}&format=json'.format( categoryF, ptype, nstype) ## append category to cat_tag
   # parameters_tag = "&prop=info|categories|links" ## return in json format
    query = base_url + action_tag + category_tag# + parameters_tag ## concatenate base_url with request tags
    return query

In [4]:
def request_elements( category, ptype, tag = False):
    query = format_query( category, ptype)
    
    r = requests.get( query)  ## request HTTP results
    response = r.json()

    elements_df = pd.DataFrame( response['query']['categorymembers'])
    if tag:
        elements_df['subcategory'] = tag 
    return elements_df
    

In [5]:
def get_subcategories( category, depth =4, subcategories = []):  ## Restrixt depth to level 4
    
    category_df = request_elements( category, 'subcat')
    for subcat in category_df.title.str.replace('Category:', '').tolist():
        
        if subcat not in subcategories:
            subcategories.append( subcat)
            try:
                if depth < 0:
                    break
                else:
                    subcategories = get_subcategories( subcat,depth - 1, subcategories)
            except:
                continue
    return subcategories
            

In [ ]:
def get_subcategories( category, depth =4, subcategories = []):  ## Restrixt depth to level 4
    
    category_df = request_elements( category, 'subcat')
    
    #cat_labels = category + "/"
    
    for subcat in category_df.title.str.replace('Category:', '').tolist():
        
        if subcat not in subcategories:
            subcategories.append( subcat)
            try:
                if depth < 0:
                    #print("BREAK")
                    break
                else:
                    subcategories = get_subcategories( subcat,depth - 1, subcategories)
            except:
                continue
    return subcategories
            

In [ ]:
category_df = request_elements( 'machine learning', 'subcat')
categories = category_df.title.str.replace('Category:', '').tolist()
categories

In [ ]:
df = request_elements( 'machine learning', 'page')

df

In [6]:
def grab_content( page_id):
    try:
        page_content = wikipedia.WikipediaPage(pageid = page_id).content
    except: 
        page_content = ''
    return page_content
    

In [14]:
def cleaner(message):
    message = re.sub('\.+', ' ', message)
    message = re.sub('[^a-z0-9 ]',' ', message.lower())
    message = re.sub('\d+',' NUMBER ',message)
    message = re.sub('\s+',' ',message)
    return message 

In [7]:
def get_pages( category, depth = 4 ):
    ##astype
    subcategories = get_subcategories( category, depth, subcategories = [] )
    
    pages_df_list = []
    #categories_pageid_dict = {}
    
    pages_df = request_elements( category, 'page', category)
    
    pages_df_list.append( pages_df)
    #categories_pageid_dict[category] = pages_df.pageid.tolist()
    
    for subcat in subcategories:
        #tag = category + "/" + subcat
        pages_df = request_elements( subcat, 'page', subcat)
        
        try:
            #categories_pageid_dict[subcat] = pages_df.pageid.tolist()
            pages_df_list.append( pages_df)
        except:
            continue
        
    pages_df = pd.concat( pages_df_list)
    
    pages_df = pages_df.reset_index(drop = True )
    
    pages_df.drop_duplicates().reset_index(drop = True, inplace = True)
    
    pages_df.drop('ns', axis = 1, inplace = True)
    
    pages_df['category'] = category
    pages_df['text'] = pages_df.pageid.apply( grab_content)
    
    
    pages_df.pageid = pages_df.pageid.apply( lambda x: int(x))
    pages_df.text = pages_df.text.apply( cleaner)
    pages_df.title = pages_df.title.apply( cleaner)
    
    return pages_df #, categories_pageid_dict
                             
        
    
    
    

In [ ]:
pd.DataFrame.reset_index()

In [ ]:
pages_df = request_elements( 'machine learning', 'page')
pages_df.pageid.tolist()

In [ ]:
ml_pages_df = get_pages( 'machine learning')
print( ml_pages_df.shape)
ml_pages_df.head()

ml_pages_df.dtypes

In [ ]:
ml_pages_df.subcategory.unique()

In [ ]:


ml_pages_df['text'] = ml_pages_df.pageid.apply( grab_content)

#pages_df['text'] = pages_df.pageid.apply( grab_content)

In [ ]:
ml_pages_df.head()

In [ ]:
!pwd

In [ ]:
ml_pages_df.to_pickle('../docker/postgres/data/ml.p')

In [8]:
bs_pages_df = get_pages( 'business software', depth = 2)
print( bs_pages_df.shape)
bs_pages_df.head()

(6545, 4)


,pageid,subcategory,title,category
0,1037763.0,business software,Business software,business software
1,41270069.0,business software,AccuSystems,business software
2,5211212.0,business software,Active policy management,business software
3,28502793.0,business software,Alexandria (library software),business software
4,44133735.0,business software,Alteryx,business software


In [ ]:
bs_pages_df.loc[:,'text'] = bs_pages_df.pageid.apply( grab_content)

In [16]:
bs_cleaned_df = bs_pages_df.copy()
bs_cleaned_df.pageid = bs_cleaned_df.pageid.apply( lambda x: int(x))
bs_cleaned_df.text = bs_cleaned_df.text.apply( cleaner)
bs_cleaned_df.title = bs_cleaned_df.title.apply( cleaner)


In [18]:
bs_cleaned_df.head()

,pageid,subcategory,title,category,text
0,1037763,business software,business software,business software,business software or a business application is...
1,41270069,business software,accusystems,business software,accusystems llc is an american company headqua...
2,5211212,business software,active policy management,business software,active policy management is business oriented ...
3,28502793,business software,alexandria library software,business software,alexandria is browser based cross platform lib...
4,44133735,business software,alteryx,business software,alteryx is an american computer software compa...


In [19]:
bs_cleaned_df.to_pickle('../docker/postgres/data/bs_cleaned.p')

## Table Functions

In [ ]:
!pip install psycopg2

import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

def connect_to_db():
    con = pg2.connect(host='postgres',
                      dbname='wikipedia',
                      user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

con, cur = connect_to_db

con = pg2.connect(host='postgres',  #127.0.0.1
                      dbname='wikipedia',
                      user='postgres')
cursor = con.cursor(cursor_factory=RealDictCursor)

In [ ]:
cursor

In [ ]:
database.py
request_categories.py

In [ ]:
ml_pages_df['title'].value_counts

In [ ]:
bs_pages_df = get_pages( 'business software', 3)
print( bs_pages_df.shape)
bs_pages_df

In [ ]:
bs_pages_df.ns.unique()

In [ ]:
def update_page_table(data):
    query = """BEGIN; INSERT INTO pages (pageid, title, article); 
        VALUES
    COMMIT;""".format()

In [ ]:
ml_cats = get_subcategories( 'machine learning', subcategories = [])

In [ ]:
ml_cats

In [ ]:
bs_cats = get_subcategories( 'business software', 3, subcategories = [])  ## Gets lost in Financial Software / 
                                                    ## Financial Markets Software / Electronic Trading Platforms /
                                                    ## Stock Exchanges          

In [ ]:
bs_cats

In [ ]:
def request_category( category ):  ##  exclude
    
    #category_query = category_format( category)
    
    base_url = 'https://en.wikipedia.org/w/api.php'
    action_tag = "?action=query&list=categorymembers&cmlimit=max" ## fetch all category members (pages, subcategories)
    category_tag = '&cmtitle=Category:{}'.format( category) ## append category to cat_tag
    parameters_tag = "&format=json&prop=info|categories|links" ## return in json format
    request_call = base_url + action_tag + category_tag + parameters_tag ## concatenate base_url with request tags

    r = requests.get( request_call)  ## request HTTP results
    response = r.json()
    
    category_pages = response['query']['categorymembers']
    
    
    category_pages_df = pd.DataFrame( response['query']['categorymembers'] )
    
    return category_pages_df
    
    
    

In [ ]:
## Original 

def gather_articles( category ):
    '''Collect all articles that belong to a category, including articles found in sub-categories.
        This process is performed recursively to find all relevant articles in the domain space'''
    print(category)
    category_pages_df = request_category( category)  ## load dataframe of pages contained in category
    
    sub_category_mask = category_pages_df.title.str.contains('Category:')  ## row mask for only sub-categories
    
    pages_df_list = []
    categories_pageid_dict = {}
    
    pages_df = category_pages_df[ ~sub_category_mask]
    pages_df_list.append( pages_df)
    
    sub_categories = category_pages_df[ sub_category_mask].title.str.replace( 'Category:', '').tolist()    ## Create list of all sub-category names ( Category: <name>), remove the preface                
    categories_pageid_dict[ category] = pages_df.pageid
    n_sub_categories = sum( sub_category_mask)  ## Number of sub-categories belong to category
    
    if n_sub_categories > 0:
        for sub_category in sub_categories:  ## recursively, one at a time     
            sub_category_pages = gather_articles( sub_category )[0]
            
            #categories_pageid_dict[ sub_category] = sub_category_pages.pageid  ## add sub 
            
            
            pages_df_list.append( sub_category_pages  )
            

    pages_df = pd.concat( pages_df_list)
    pages_df.reset_index()
    return pages_df, categories_pageid_dict
    
    
    #pages_df_list.append(  category_pages_df[~sub_category_mask].pageid.apply( grab_content ))
    
    